In [157]:
from scipy.io import loadmat
import numpy as np

In [176]:
def sigmoid(x):
    return 1.0/(1+np.exp(-x))

def sigmoid_derivative(x):
    return x*(1.0-x)

def softmax(x):
    ex = np.exp(x)
    return ex/np.sum(ex, axis=1, keepdims=True)

In [159]:
# Part (b)
ion_train = loadmat("Ion.trin.mat")
ion_test = loadmat("Ion.test.mat")

In [160]:
tst = np.zeros((4,1))
tst[2][0]

0.0

In [177]:
# Part (a)
class SingleLayerNN:
    def __init__(self, X, y, num_classes = 0, 
                 hidden = 0, decay = 0.0, epochs = 50, 
                 seed = 314152, l2 = 0.01, learning_rate = 0.1):
        self.X = X
        self.y = y
        self.epochs = epochs
        self.seed = seed
        self.l2 = l2
        self.lrate = learning_rate
        self.decay = 0.0
    
        d, n = X.shape
        if hidden == 0:
            self.hidden = n + 1
        else:
            self.hidden = hidden
            
        if num_classes == 0:
            self.k = len(np.unique(y))
        else:
            self.k = num_classes
            
        # Setting a seed to get consistent results when testing
        np.random.seed(self.seed)

        # Initializing random weights and biases
        self.w = np.random.randn(d, self.hidden)/np.sqrt(d)
        self.b = np.zeros((1, self.hidden))
        ## output layer weights/biases
        self.wy = np.random.randn(self.hidden, self.k)/np.sqrt(self.hidden)
        self.by = np.zeros((1,self.k))
        
        # Storing the activations
        self.hidden_activation = np.zeros((n, self.hidden))
        self.output = np.zeros((n, self.k))
        
    def feed_forward(self, input_set):
        hidden_layer = np.dot(input_set.T, self.w) + self.b
        self.hidden_activation = np.tanh(hidden_layer)
        output_layer = np.dot(self.hidden_activation, self.wy) + self.by
        output_activation = softmax(output_layer)
        return output_activation
    
    def fit(self):
        error_rates = [0] * self.epochs
        error = 0.0
        
        for i in range(self.epochs):
            # Shuffling the dataset
            d, n = self.X.shape
            dataset = np.append(self.X, self.y.T, axis = 0)
            np.random.shuffle(dataset.T)
            input_set = dataset[:d]
            targets = np.reshape(dataset[d], (n,1)).astype(int)
            self.output = self.feed_forward(input_set)
            
            # Backpropagation
            delta = self.output
            delta[range(n), targets] = 1 - delta[range(n), targets]
            output_delta = np.dot(self.hidden_activation.T, delta)
            delta2 = np.dot(delta, self.wy.T) * (1 - np.power(self.hidden_activation, 2))
            hidden_delta = np.dot(self.X, delta2)
            
            ## Regularization
            hidden_reg = self.l2 * self.w
            output_reg = self.l2 * self.wy
            ## Update weights
            self.wy -= self.lrate * (output_delta + output_reg)
            self.w -= self.lrate * (hidden_delta + hidden_reg)
            ## Update biases
            self.by -= self.lrate * np.sum(output_delta, axis=0, keepdims=True) 
            self.b -= self.lrate * np.sum(hidden_delta, axis=0, keepdims=True)
            self.lrate *= 1.0 /(1 + self.decay*i)

        return error_rates
    
    def predict(self, data):
        probs = self.feed_forward(data)
        return np.argmax(probs, axis=1)

In [187]:
model = SingleLayerNN(ion_train['Xtrain'], ion_train['ytrain'],
                      hidden = 360, epochs=10000, decay = 0.01, l2=0.01)
errors = model.fit()

/Users/azoachaforcheh/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':


In [188]:
predictions = model.predict(ion_test['Xtest'])

In [189]:
n = len(ion_test['ytest'])
correct = 0
for i in range(n):
    if ion_test['ytest'][i] == predictions[i]:
        correct +=1
1.0*correct/n

0.64